In [1]:
import glob
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib qt 

In [2]:
"""
***Aestetics***
"""
# load color palette
my_palette = sns.color_palette().as_hex()
sns.color_palette().as_hex()

['#1f77b4',
 '#ff7f0e',
 '#2ca02c',
 '#d62728',
 '#9467bd',
 '#8c564b',
 '#e377c2',
 '#7f7f7f',
 '#bcbd22',
 '#17becf']

In [31]:
""" 
Load files
"""
el_1 = "PO4"
el_2 = "PO3"
el_3 = "PO8"
el_4 = "PO7"
# el_2_path = "C:/Users/mvmigem/Documents/data/project_1/overlap_corrected/variable-electrode/"
el_1_path = f"C:/Users/mvmigem/Documents/data/project_1/overlap_corrected/{el_1}/"
el_2_path = f"C:/Users/mvmigem/Documents/data/project_1/overlap_corrected/{el_2}/"
el_3_path = f"C:/Users/mvmigem/Documents/data/project_1/overlap_corrected/{el_3}/"
el_4_path = f"C:/Users/mvmigem/Documents/data/project_1/overlap_corrected/{el_4}/"

el_1_dir_list = glob.glob(el_1_path+'/*.csv')
el_2_dir_list = glob.glob(el_2_path+'/*.csv')
el_3_dir_list = glob.glob(el_3_path+'/*.csv')
el_4_dir_list = glob.glob(el_4_path+'/*.csv')
excuded_pp = [3,14,20]
subject_list = []

for i in el_1_dir_list:
    sub = int(i.split(f'corrected_{el_1}_evoked_')[1].split('.csv')[0])
    if sub not in excuded_pp:
        subject_list.append(sub)

# Create big df for each var and fix set of rERP's
el_1_loaded_dfs = []
el_2_loaded_dfs = []
el_3_loaded_dfs = []
el_4_loaded_dfs = []

for el1_p,el2_p,el3_p,el4_p in zip(el_1_dir_list,el_2_dir_list,el_3_dir_list,el_4_dir_list):
    el_1_rerp_df = pd.read_csv(el1_p)
    el_1_loaded_dfs.append(el_1_rerp_df)
    el_2_rerp_df = pd.read_csv(el2_p)
    el_2_loaded_dfs.append(el_2_rerp_df)
    el_3_rerp_df = pd.read_csv(el3_p)
    el_3_loaded_dfs.append(el_3_rerp_df)
    el_4_rerp_df = pd.read_csv(el4_p)
    el_4_loaded_dfs.append(el_4_rerp_df)

df_1 = pd.concat(el_1_loaded_dfs,ignore_index=True)
df_2 = pd.concat(el_2_loaded_dfs,ignore_index=True)
df_3 = pd.concat(el_3_loaded_dfs,ignore_index=True)
df_4 = pd.concat(el_4_loaded_dfs,ignore_index=True)

df_1['visual_field'] = np.where(df_1['position'].isin([1,2]),'up','down')
df_2['visual_field'] = np.where(df_2['position'].isin([1,2]),'up','down')
df_3['visual_field'] = np.where(df_3['position'].isin([1,2]),'up','down')
df_4['visual_field'] = np.where(df_4['position'].isin([1,2]),'up','down')

df_1 = df_1[df_1['time'] < .5]
df_2 = df_2[df_2['time'] < .5]
df_3 = df_3[df_3['time'] < .5]
df_4 = df_4[df_4['time'] < .5]

df_1.reset_index(inplace=True)
df_2.reset_index(inplace=True)
df_3.reset_index(inplace=True)
df_4.reset_index(inplace=True)


In [34]:
""" 
baseline correction
"""
for df in [df_1,df_2,df_3,df_4]:
    baseline_period = df['time'] <0
    baseline_means = df[baseline_period].groupby(['subject','attention','expectation','visual_field'])['yhat'].mean()

    df['baseline_corrected'] = df.apply(
        lambda row: row['yhat'] - baseline_means[(row['subject'], row['attention'], row['expectation'],row['visual_field'])],
        axis=1
    )
    # Correct dataframe for subjects 8 and 9 having truncated datasets (lost trials)
    df.loc[(df['subject'] == 9) & (df['attention'] == 'attended') & (df['expectation'] == 'odd'), 
        ['yhat', 'baseline_corrected']] = np.nan
    df.loc[(df['subject'] == 9) & (df['attention'] == 'attended') & (df['expectation'] == 'regular'), 
        ['yhat', 'baseline_corrected']] = np.nan
    df.loc[(df_1['subject'] == 8) & (df['attention'] == 'attended') & (df['expectation'] == 'regular'), 
        ['yhat', 'baseline_corrected']] = np.nan
    df.loc[(df['subject'] == 8) & (df['attention'] == 'attended') & (df['expectation'] == 'odd'), 
        ['yhat', 'baseline_corrected']] = np.nan

In [30]:
fig, axes = plt.subplots(2,2,figsize=(16, 10),sharey=True,sharex=True)

sns.lineplot(data=df_1,
             x='time',y='baseline_corrected',
             hue='attention',style='expectation',
             errorbar='se',ax=axes[0,0])
sns.lineplot(data=df_1[df_1['visual_field']=='up'],
             x='time',y='baseline_corrected',
             hue='attention',style='expectation',
             errorbar='se', ax = axes[1,0])
sns.lineplot(data=df_2,
             x='time',y='baseline_corrected',
             hue='attention',style='expectation',
             errorbar='se',ax=axes[0,1])
sns.lineplot(data=df_2[df_2['visual_field']=='up'],
             x='time',y='baseline_corrected',
             hue='attention',style='expectation',
             errorbar='se', ax = axes[1,1])
axes[0,0].set_title(f'{el_1}')
axes[0,1].set_title(f'{el_2}')
plt.tight_layout()
plt.show()

In [ ]:
""" 
Select the C1 to analyse
"""
##  C1 time window
# Window search
c1_tmin = 0.06
c1_tmax = 0.09
c1_timwid = 0.015
c1_sampwid = int(c1_timwid*512)
for df in [df_1,df_2,df_3,df_4]:
# Find indices of df with peak in window DF 1
slect = df_1[(df_1['time']>c1_tmin) & (df_1['time']<c1_tmax)]
slect['abs_corr'] = slect['baseline_corrected'].abs()
maxidx_c1 = np.array(slect.groupby(['subject','attention','expectation','visual_field'])['abs_corr'].idxmax())
maxidx_c1 = maxidx_c1[~np.isnan(maxidx_c1)]
maxidx_c1 = maxidx_c1.astype(int)
# Select window around it
winidx_c1 =[]
for i, idx in enumerate(maxidx_c1):
    win_li = np.arange(idx - c1_sampwid, idx + c1_sampwid +1)
    winidx_c1 = np.concatenate([winidx_c1,win_li],axis=0)

# Apply to df
c1_win_df_1 = df_1.iloc[winidx_c1]
c1_means_df_1 = c1_win_df_1.groupby(['subject','attention','expectation','visual_field'])['baseline_corrected'].mean().reset_index()


# Apply to df
c1_win_df_2 = df_2.iloc[winidx_c1]
c1_means_df_2 = c1_win_df_2.groupby(['subject','attention','expectation','visual_field'])['baseline_corrected'].mean().reset_index()

In [ ]:
""" 
Select P3 to analyse
"""



In [ ]:
destination_folder = r"C:\Users\mvmigem\Documents\data\project_1\compiled_dataframes/"
c1_means_df_2.to_csv(destination_folder + "c1_oc_var_df.csv")